In [ ]:

def get_index_comma(string):
    """
    Return index of commas in string

    """
    index_list = list()
    # Count open parentheses
    par_count = 0

    for i in range(len(string)):
        if string[i] == ',' and par_count == 0:
            index_list.append(i)
        elif string[i] == '(':
            par_count += 1
        elif string[i] == ')':
            par_count -= 1

    return index_list

def is_variable(expr):
    """
    Check if expression is variable
    """
    for i in expr:
        if i == '(':
            return False
    return True

def process_expression(expr):
    """
    input:  - expression:
            'Q(a, g(x, b), f(y))'
    return: - predicate symbol:
            Q
            - list of arguments
            ['a', 'g(x, b)', 'f(y)']
    """
    # Remove space in expression
    expr = expr.replace(' ', '')

    # Find the first index == '('
    index = None
    for i in range(len(expr)):
        if expr[i] == '(':
            index = i
            break

    # Return predicate symbol and remove predicate symbol in expression
    predicate_symbol = expr[:index]
    expr = expr.replace(predicate_symbol, '')

    # Remove '(' in the first index and ')' in the last index
    expr = expr[1:len(expr) - 1]

    # List of arguments
    arg_list = list()

    # Split string with commas, return list of arguments
    indices = get_index_comma(expr)

    if len(indices) == 0:
        arg_list.append(expr)
    else:
        arg_list.append(expr[:indices[0]])
        for i, j in zip(indices, indices[1:]):
            arg_list.append(expr[i + 1:j])
        arg_list.append(expr[indices[len(indices) - 1] + 1:])

    return predicate_symbol, arg_list

def get_arg_list(expr):
    """
    input:  expression:
            'Q(a, g(x, b), f(y))'
    return: full list of arguments:
            ['a', 'x', 'b', 'y']
    """
    _, arg_list = process_expression(expr)

    flag = True
    while flag:
        flag = False

        for i in arg_list:
            if not is_variable(i):
                flag = True
                _, tmp = process_expression(i)
                for j in tmp:
                    if j not in arg_list:
                        arg_list.append(j)
                arg_list.remove(i)

    return arg_list

def check_occurs(var, expr):
    """
    Check if var occurs in expr
    """
    arg_list = get_arg_list(expr)
    if var in arg_list:
        return True

    return False

def unify(expr1, expr2):
    """
    Unification Algorithm
    Step 1: If Ψ1 or Ψ2 is a variable or constant, then:
              a, If Ψ1 or Ψ2 are identical, then return NULL.
              b, Else if Ψ1 is a variable:
                  - then if Ψ1 occurs in Ψ2, then return False
                  - Else return (Ψ2 / Ψ1)
              c, Else if Ψ2 is a variable:
                  - then if Ψ2 occurs in Ψ1, then return False
                  - Else return (Ψ1 / Ψ2)
              d, Else return False
    Step 2: If the initial Predicate symbol in Ψ1 and Ψ2 are not same, then return False.
    Step 3: IF Ψ1 and Ψ2 have a different number of arguments, then return False.
    Step 4: Create Substitution list.
    Step 5: For i=1 to the number of elements in Ψ1.
              a, Call Unify function with the ith element of Ψ1 and ith element of Ψ2, and put the result into S.
              b, If S = False then returns False
              c, If S ≠ Null then append to Substitution list
    Step 6: Return Substitution list.
    """
    # Step 1:
    if is_variable(expr1) and is_variable(expr2):
        if expr1 == expr2:
            return 'Null'
        else:
            return False
    elif is_variable(expr1) and not is_variable(expr2):
        if check_occurs(expr1, expr2):
            return False
        else:
            tmp = str(expr2) + '/' + str(expr1)
            return tmp
    elif not is_variable(expr1) and is_variable(expr2):
        if check_occurs(expr2, expr1):
            return False
        else:
            tmp = str(expr1) + '/' + str(expr2)
            return tmp
    else:
        predicate_symbol_1, arg_list_1 = process_expression(expr1)
        predicate_symbol_2, arg_list_2 = process_expression(expr2)

        # Step 2
        if predicate_symbol_1 != predicate_symbol_2:
            return False
        # Step 3
        elif len(arg_list_1) != len(arg_list_2):
            return False
        else:
            # Step 4: Create substitution list
            sub_list = list()

            # Step 5:
            for i in range(len(arg_list_1)):
                tmp = unify(arg_list_1[i], arg_list_2[i])

                if not tmp:
                    return False
                elif tmp == 'Null':
                    pass
                else:
                    if type(tmp) == list:
                        for j in tmp:
                            sub_list.append(j)
                    else:
                        sub_list.append(tmp)

            # Step 6
            return sub_list

if __name__ == '__main__':
    # Data 1
    f1 = 'p(b(A), X, f(g(Z)))'
    f2 = 'p(Z, f(Y), f(Y))'

    # Data 2
    #f1 = 'Q(a, g(x, a), f(y))'
    #f2 = 'Q(a, g(f(b), a), x)'

    # Data 3
    #f1 = 'Q(a, g(x, a, d), f(y))'
    #f2 = 'Q(a, g(f(b), a), x)'

    result = unify(f1, f2)
    if not result:
        print('Unification failed!')
    else:
        print('Unification successfully!')
        print(result)

Unification successfully!
['b(A)/Z', 'f(Y)/X', 'g(Z)/Y']
